In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np


In [2]:
import cv2
import csv
import mediapipe as mp


mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

def datapoints():
    cap = cv2.VideoCapture(0)#open webcam
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 500) #set frame size
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    gesture_name = input("Enter gesture: ")  #creating label
    count = 1  # number of coords
    datapoint = []  # holds the same gesture but different landmarks for each
    
    while True:  # Continuous frames
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to read frame.")
            break
        
        frame = cv2.flip(frame, 1)  # mirroring
        y, x, channel = frame.shape
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # convert BGR to RGB
        result = hands.process(img)  # This is the result of the hand mediapipe model
        landmarks = {'gesture': gesture_name}  # Initializing a row for a dataframe

        if result.multi_hand_landmarks:  # we will get a list of dictionaries with each landmark as key and x,y coords as value
            for handslms in result.multi_hand_landmarks:
                mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

                lndmarks = handslms.landmark#this is my landmark
                for i in range(len(lndmarks)):
                    landmarks[f'x{i}'] = lndmarks[i].x * x#corresponding values of coordinates to that landmark
                    landmarks[f'y{i}'] = lndmarks[i].y * y#multiplication to scale these values like 0.5*500=250 pixel (middle screen)
        
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1)
        
        if key == ord("q"):  # Breaks input
            break
        elif key == ord("s"):  # Saves a single frame
            datapoint.append(landmarks)
            print(f'{count} {landmarks}')
            count += 1
    
    cap.release()
    cv2.destroyAllWindows()

    # save the collected data as a CSV file
    if datapoint:
        with open(f'{gesture_name}_data.csv', mode='w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=datapoint[0].keys())
            writer.writeheader()
            writer.writerows(datapoint)
            print(f"Data saved to {gesture_name}_data.csv")
    
    return datapoint







I0000 00:00:1737234013.219259  447708 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1


In [3]:
data=datapoints()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1737234013.234361  447788 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737234013.243803  447788 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737234019.963049  447789 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-01-19 02:30:20.230 Python[6332:447708] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-19 02:30:20.230 Python[6332:447708] +[IMKInputSession subclass]: chose IMKInputSession_Modern


1 {'gesture': 'Forward', 'x0': 109.27091598510742, 'y0': 144.0662670135498, 'x1': 137.02752113342285, 'y1': 156.47912979125977, 'x2': 158.4091091156006, 'y2': 174.59279537200928, 'x3': 159.0275478363037, 'y3': 194.94160652160645, 'x4': 147.8950309753418, 'y4': 206.06476306915283, 'x5': 155.8687686920166, 'y5': 144.17118072509766, 'x6': 162.88854598999023, 'y6': 153.7649631500244, 'x7': 163.5166358947754, 'y7': 152.23126888275146, 'x8': 160.33281326293945, 'y8': 149.42087173461914, 'x9': 138.86545181274414, 'y9': 137.464599609375, 'x10': 141.92540168762207, 'y10': 179.01537895202637, 'x11': 140.13761520385742, 'y11': 180.5444097518921, 'x12': 138.42917442321777, 'y12': 176.7027711868286, 'x13': 119.29888725280762, 'y13': 139.68114852905273, 'x14': 122.92789459228516, 'y14': 185.94367504119873, 'x15': 123.21786880493164, 'y15': 179.18470859527588, 'x16': 123.15182685852051, 'y16': 168.98227214813232, 'x17': 101.28283500671387, 'y17': 146.65969848632812, 'x18': 108.15329551696777, 'y18': 

In [4]:
df=pd.dataframe(data)
df.head()


AttributeError: module 'pandas' has no attribute 'dataframe'

In [1]:
old_df = pd.read_csv('gestures.csv')


NameError: name 'pd' is not defined

In [ ]:
new_df = pd.concat([old_df, df], ignore_index=True)
new_df.to_csv('gestures.csv', index=False)